In [3]:
import zipfile
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image, ImageEnhance
import random
import shutil
import cv2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import label_binarize

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
zip_path = "/content/drive/MyDrive/FODS PROJECT FOLDER/German Dataset.zip"
extract_path = "GTS"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Files extracted to:", extract_path)

Files extracted to: GTS


In [6]:
german_base = "GTS"

german_train_csv = os.path.join(german_base, "Train.csv")
german_train_img_base = os.path.join(german_base, "Train")

german_test_csv = os.path.join(german_base, "Test.csv")
german_test_img_base = os.path.join(german_base, "Test")



ge_train_df = pd.read_csv(german_train_csv)
ge_test_df = pd.read_csv(german_test_csv)


print("German Train CSV shape:", ge_train_df.shape)
print("German Test CSV shape:", ge_test_df.shape)



German Train CSV shape: (39209, 8)
German Test CSV shape: (12630, 8)


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

# ============================================================
# 1️⃣ PATHS
# ============================================================
train_dir = "/content/GTS/Train"
test_dir  = "/content/GTS/Test"
test_csv  = "/content/GTS/Test.csv"

# ============================================================
# 2️⃣ DATA PIPELINE (Optimized)
# ============================================================

IMG_SIZE = (96, 96)
BATCH_SIZE = 32

# GPU-based data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Train/validation datasets using image_dataset_from_directory
train_ds_raw = tf.keras.utils.image_dataset_from_directory(
    directory=train_dir,
    validation_split=0.15,
    subset='training',
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds_raw = tf.keras.utils.image_dataset_from_directory(
    directory=train_dir,
    validation_split=0.15,
    subset='validation',
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

num_classes = len(train_ds_raw.class_names)
print("Detected classes:", num_classes)

# Normalize pixel values (0–1)
normalization_layer = layers.Rescaling(1./255)

# Prefetch and cache for GPU efficiency
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds_raw.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds_raw.cache().prefetch(buffer_size=AUTOTUNE)

# ============================================================
# 3️⃣ BUILD MODEL (MobileNetV2)
# ============================================================

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
base_model.trainable = False  # Freeze base layers

model = models.Sequential([
    data_augmentation,
    normalization_layer,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# ============================================================
# 4️⃣ TRAIN
# ============================================================

lr_cb = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2)
ckpt_cb = ModelCheckpoint('best_mobilenet_model.keras', save_best_only=True, monitor='val_accuracy', mode='max')
es_cb   = EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)

history = model.fit(
    train_ds,
    epochs=25,
    validation_data=val_ds,
    callbacks=[lr_cb, ckpt_cb, es_cb],
    verbose=1
)

# ============================================================
# 5️⃣ OPTIONAL FINE-TUNING
# ============================================================

base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

fine_history = model.fit(
    train_ds,
    epochs=25,
    validation_data=val_ds,
    callbacks=[lr_cb, ckpt_cb, es_cb],
    verbose=1
)

Found 39209 files belonging to 43 classes.
Using 33328 files for training.
Found 39209 files belonging to 43 classes.
Using 5881 files for validation.
Detected classes: 43


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_96             │ (None, 3, 3, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/25
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 55s 25ms/step - accuracy: 0.3464 - loss: 2.3214 - val_accuracy: 0.6475 - val_loss: 1.0974 - learning_rate: 0.0010
Epoch 2/25
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.5423 - loss: 1.3994 - val_accuracy: 0.6931 - val_loss: 0.9421 - learning_rate: 0.0010
Epoch 3/25
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.5833 - loss: 1.2493 - val_accuracy: 0.7286 - val_loss: 0.8247 - learning_rate: 0.0010
Epoch 4/25
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - accuracy: 0.6140 - loss: 1.1605 - val_accuracy: 0.7594 - val_loss: 0.7586 - learning_rate: 0.0010
Epoch 5/25
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.6320 - loss: 1.0993 - val_accuracy: 0.7606 - val_loss: 0.7088 - learning_rate: 0.0010
Epoch 6/25
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.6345 - loss: 1.0783 - val_accuracy: 0.7796 - val_loss: 0.6721 - learning_rate: 0.0010
Epoch 7/25
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accura